In [1]:
import pandas as pd
import datetime as dt

In [2]:
tablero_vacantes= pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\Actualizacion de informacion\Vacantes 2025.xlsx")
                            
tablero_aps = pd.read_excel(r"C:\Users\centro.control\Downloads\TableroAps-2025-07-16.xlsx", skiprows=[1])
nuevos_nombres = pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\Sitios SAP.xlsx")

In [3]:
criterio_aps =(pd.to_datetime(tablero_aps['FECHA'].max())).date()
criterio_tablero =  (tablero_vacantes['Fecha del reporte'].max()).date()


In [4]:
nombre_sap = nuevos_nombres[['SITIO','Nombre Comercial']]

In [5]:
tablero_aps = tablero_aps.merge(nombre_sap, on='SITIO', how='left')
tablero_aps['SITIO'] = tablero_aps['Nombre Comercial']
tablero_aps= tablero_aps.drop(columns=['Nombre Comercial'])

In [6]:
nuevos_nombres = nuevos_nombres.rename(columns={
    'Nombre Comercial': 'Sitio'
})

In [7]:
def tabla_turno(turno_str):
    # Determinar índice y nombre de columna
    if turno_str == 'Matutino':
        i = 6
        aux = 'MAT'
        turno= 'Matutino'

    elif turno_str == 'Vespertino':
        i = 9
        aux = 'Vesp'
        turno= 'Vespertino'

    elif turno_str == 'Nocturno':
        i = 12
        aux = 'Noc'
        turno= 'Nocturno'

    elif turno_str == 'Mixto':
        i = 15
        aux = 'Mix'
        turno= 'Mixto'

    else:
        raise ValueError("Turno no válido. Usa 'Matutino', 'Vespertino', 'Nocturno' o 'Mixto'.")

    # Selección y renombramiento de columnas
    columnas = ['FECHA', 'SITIO', aux, f'Unnamed: {i}', f'Unnamed: {i+1}']
    turno_df = tablero_aps[columnas].copy()

    turno_df = turno_df.rename(columns={
        'FECHA': 'Fecha del reporte',
        'SITIO': 'Sitio',
        aux: 'Plantilla',
        f'Unnamed: {i}': 'Activos',
        f'Unnamed: {i+1}': 'Vacante'
    })

    turno_df['Turno'] = turno
    turno_df['Tipo'] = 'Natural'
    turno_df['Ejecutivo'] = ''

    return turno_df


def cruzar_id(turno):
    turno = turno.merge(nuevos_nombres ,on='Sitio', how='left').drop(['SAP', 'Razón Social OK', 'RFC'], axis=1).fillna(0)
    turno['id'] = turno['id'].astype(int) 
    turno = turno[['id','Fecha del reporte', 'Sitio', 'Plantilla', 'Activos', 'Vacante','Turno', 'Tipo', 'Ejecutivo']]
    return turno

In [8]:
Matutino = tabla_turno('Matutino')
Vespertino = tabla_turno('Vespertino')
Nocturno = tabla_turno('Nocturno')
Mixto = tabla_turno('Mixto')

In [9]:
Matutino = cruzar_id(Matutino)
Vespertino = cruzar_id(Vespertino)
Nocturno = cruzar_id(Nocturno)
Mixto = cruzar_id(Mixto)

In [10]:
tablero_actualizado = pd.concat([tablero_vacantes, Matutino, Vespertino, Nocturno, Mixto], ignore_index=True)

In [11]:
tablero_actualizado['Fecha del reporte'] = pd.to_datetime(tablero_actualizado['Fecha del reporte'])
tablero_actualizado['Fecha del reporte'] = tablero_actualizado['Fecha del reporte'].dt.date
tablero_actualizado = tablero_actualizado.drop_duplicates()

In [12]:
if criterio_aps == criterio_tablero: 
        print('No se pudo actualizar la información')
else:
    tablero_actualizado.to_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\Actualizacion de informacion\Vacantes 2025.xlsx", index=False)